In [1]:
from transformers import BertTokenizer, BertForTokenClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, Dataset
import torch
import json
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import json
with open('/kaggle/input/train-json/Subtask_1_train.json', 'r') as file:
    data = json.load(file)

In [3]:
conversation_list = []

for i in data:
    
    temp = i['conversation']
    text_list = []
    for j in temp:
        text_list.append(j['text'])
    
    emotion_cause = i['emotion-cause_pairs']
    ec_dict = {}
    for j in emotion_cause:
        if j[0] not in ec_dict:
            ec_dict[j[0]] = [j[1]]
        else:
            ec_dict[j[0]].append(j[1])
            
    conversation_list.append({'text':text_list,'emotion_cause': ec_dict})
    
print(len(conversation_list))

1374


In [4]:
for i in conversation_list:
    print(i)
    break

{'text': ['Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .', 'Oh , yeah . Had that dream .', 'Then I look down , and I realize there is a phone ... there .', 'Instead of ... ?', 'That is right .', 'Never had that dream .', 'No .', 'All of a sudden , the phone starts to ring .'], 'emotion_cause': {'3_surprise': ['1_I realize I am totally naked .', '3_Then I look down , and I realize there is a phone ... there .'], '4_surprise': ['1_I realize I am totally naked .', '3_Then I look down , and I realize there is a phone ... there .', '4_Instead of ...'], '5_anger': ['1_I realize I am totally naked .', '3_Then I look down , and I realize there is a phone ... there .', '4_Instead of ...']}}


In [5]:
def finder(list1, list2):
    i = 0
    j = 0
    while i<len(list1):
        if(list1[i]==list2[j]):
            j+=1
        else:
            j=0
        if(j == len(list2)):
            return i-j+1, i+1
        i+=1
    return -1, -1

# print(finder([1,24,223,4,23,5,342,12,42],[223,4,23,5]))

In [6]:
emotion_list = ['anger','joy','fear','disgust','surprise','sadness','overall']
data_dict = {}
for i in emotion_list:
    data_dict[i] = []
    
for i in conversation_list:
    
    text_list = i['text']
    emotion_list = i['emotion_cause']
    
    for j in emotion_list:
        
        data_item = {}
        temp = j.split("_")
        idx=int(j[0])
        emotion= temp[1]
        emotion_cause_list = emotion_list[j]
        
        text= []
        label = []
        
        for k in range(idx):
            temp_text = text_list[k].split()
            temp_label = [0]*len(temp_text)
            
            text.extend(temp_text)
            label.extend(temp_label)
            
        for k in emotion_cause_list:
            
            temp = k.split("_")[1].split()
            s_idx, l_idx = finder(text, temp)

            for x in range(s_idx,l_idx):
                label[x] = 1

        data_item['text'] = text
        data_item['label'] = label
        data_dict[emotion].append(data_item)
        data_item['text'] = [emotion,':'] + text
        data_item['label'] = [1,0] + label
        data_dict['overall'].append(data_item)

In [7]:
for i in data_dict['surprise']:
    print(i)
    break

{'text': ['surprise', ':', 'Alright', ',', 'so', 'I', 'am', 'back', 'in', 'high', 'school', ',', 'I', 'am', 'standing', 'in', 'the', 'middle', 'of', 'the', 'cafeteria', ',', 'and', 'I', 'realize', 'I', 'am', 'totally', 'naked', '.', 'Oh', ',', 'yeah', '.', 'Had', 'that', 'dream', '.', 'Then', 'I', 'look', 'down', ',', 'and', 'I', 'realize', 'there', 'is', 'a', 'phone', '...', 'there', '.'], 'label': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
emotion_list = ['anger','joy','fear','disgust','surprise','sadness','overall']

for i in emotion_list:
    print(i)
    print(len(data_dict[i]))

anger
1373
joy
2093
fear
255
disgust
375
surprise
1624
sadness
1041
overall
6761


In [9]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = self.data[idx]['label']
        label = label + [0]*(512-len(label))
        text_str = ""   
        for k in text:
            text_str+=k
            text_str+=" "
        inputs = self.tokenizer(text_str, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        inputs['input_ids'] = inputs['input_ids'].reshape([512])
        inputs['token_type_ids'] = inputs['token_type_ids'].reshape([512])
        inputs['attention_mask'] = inputs['attention_mask'].reshape([512])
        
        return [inputs, torch.tensor(label, dtype=torch.long)]

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
train_loader_dict = {}
val_loader_dict = {}

for i in emotion_list:
    train_data, val_data = train_test_split(data_dict[i], test_size=0.2, random_state=42)
    train_loader_dict[i] = DataLoader(CustomDataset(train_data, tokenizer), batch_size=16, shuffle=True)
    val_loader_dict[i]= DataLoader(CustomDataset(val_data, tokenizer), batch_size=2, shuffle=False)

In [12]:
learning_rate = 2e-5
emotion_list = ['anger','joy','fear','disgust','surprise','sadness','overall']
model_dict = {}
optimizer_dict = {}
for i in emotion_list:
    temp_model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=2).to(device)
    temp_optimizer = optimizer = torch.optim.AdamW(temp_model.parameters(), lr=learning_rate)
    model_dict[i] = temp_model
    optimizer_dict[i] = temp_optimizer

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: 

In [12]:
from sklearn.metrics import f1_score
model = model_dict['sadness']
optimizer = optimizer_dict['sadness']
train_loader = train_loader_dict['sadness']
val_loader = val_loader_dict['sadness']
                             
max_f1 = 0

epochs = 15
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    collated_total = []
    predicted_total = []

    with torch.no_grad():
        for batch in val_loader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            outputs = model(**inputs, labels=labels)
            loss, scores = outputs[:2]
            loss = outputs.loss
            val_loss += loss.item()
            predicted = outputs.logits.argmax(dim=-1)
            collated_total.extend(labels.flatten().tolist())
            predicted_total.extend(predicted.flatten().tolist())

    avg_val_loss = val_loss / len(val_loader)
    f1 = f1_score(collated_total, predicted_total)
    if(f1>max_f1):
        max_f1 = f1
        torch.save(model.state_dict(), 'sadness_model.pth')
    print(f'Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, F1: {f1}')

Epoch 1/15, Average Training Loss: 0.0801
Epoch 1/15, Validation Loss: 0.0358, F1: 0.26973684210526316
Epoch 2/15, Average Training Loss: 0.0370
Epoch 2/15, Validation Loss: 0.0311, F1: 0.5231158961367954
Epoch 3/15, Average Training Loss: 0.0316
Epoch 3/15, Validation Loss: 0.0292, F1: 0.46759085799925065
Epoch 4/15, Average Training Loss: 0.0273
Epoch 4/15, Validation Loss: 0.0362, F1: 0.5921296296296296
Epoch 5/15, Average Training Loss: 0.0223
Epoch 5/15, Validation Loss: 0.0311, F1: 0.605890603085554
Epoch 6/15, Average Training Loss: 0.0184
Epoch 6/15, Validation Loss: 0.0344, F1: 0.5985699693564862
Epoch 7/15, Average Training Loss: 0.0141
Epoch 7/15, Validation Loss: 0.0364, F1: 0.6246169560776303
Epoch 8/15, Average Training Loss: 0.0118
Epoch 8/15, Validation Loss: 0.0371, F1: 0.6158881285331748
Epoch 9/15, Average Training Loss: 0.0097
Epoch 9/15, Validation Loss: 0.0415, F1: 0.6241664443851694
Epoch 10/15, Average Training Loss: 0.0089
Epoch 10/15, Validation Loss: 0.0449, 

In [13]:
from sklearn.metrics import f1_score
model = model_dict['surprise']
optimizer = optimizer_dict['surprise']
train_loader = train_loader_dict['surprise']
val_loader = val_loader_dict['surprise']
state_dict = torch.load('/kaggle/input/surprise_model/pytorch/surprise/1/surprise_model.pth')
model.load_state_dict(state_dict)
                             
max_f1 = 0

model.eval()
val_loss = 0
correct = 0
total = 0
collated_total = []
predicted_total = []

with torch.no_grad():
    for batch in val_loader:
        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        outputs = model(**inputs, labels=labels)
        loss, scores = outputs[:2]
        loss = outputs.loss
        val_loss += loss.item()
        predicted = outputs.logits.argmax(dim=-1)
        collated_total.extend(labels.flatten().tolist())
        predicted_total.extend(predicted.flatten().tolist())

avg_val_loss = val_loss / len(val_loader)
f1 = f1_score(collated_total, predicted_total)
print(f'Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, F1: {f1}')

RuntimeError: Error(s) in loading state_dict for BertForTokenClassification:
	size mismatch for bert.embeddings.word_embeddings.weight: copying a param with shape torch.Size([30522, 768]) from checkpoint, the shape in current model is torch.Size([28996, 768]).

In [ ]:
def text_tokenizer(text, label, tokenizer):
    label = label + [0]*(512-len(label))
    text_str = ""   
    for k in text:
        text_str+=k
        text_str+=" "
    inputs = tokenizer(text_str, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    return [inputs, torch.tensor(label, dtype=torch.long)]

In [ ]:
def idx_returner(list1):
    ans = []
    s_idx = -1
    l_idx = -1
    for i, item in enumerate(list1):
        if item==1 and s_idx==-1:
            s_idx = i
        if item==1 and s_idx!=-1:
            l_idx = i
        if item==0 and l_idx!=-1:
            ans.append([s_idx+2,l_idx+2])
            s_idx=-1
            l_idx=-1
    if(s_idx!=-1 and l_idx!=-1):
        ans.append([s_idx+2,l_idx+2])
    print(ans)
            

In [ ]:
for i in data_dict['overall'][52:]:
    text_list = i['text']
    true_label = i['label']
    inputs, _ = text_tokenizer(text_list, true_label, tokenizer)
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted = outputs.logits.argmax(dim=-1)
    final_predicted = predicted.flatten().tolist()[:len(text_list)]
    print(final_predicted)
    print(true_label)
    emotion = text_list[0]
    idx_returner(final_predicted[2:])
    break